In [1]:
from bs4 import BeautifulSoup as soup
import requests as req
import pandas as pd
req_url = req.get('https://www.imdb.com/search/title?title_type=feature&num_votes=25000,&genres=action')
Content = req_url.content

page = soup(Content,'html.parser')

#print(soup.prettify(movies_container[0]))

movies_container = page.findAll('div',{'class':'lister-item-content'})
print(len(movies_container))

50


In [2]:
#print(soup.prettify(movies_container[0]))

In [3]:
'''votes ={}
gross = {}
v_and_g = movies_container[0].find(class_='sort-num_votes-visible').get_text(strip=True)
x= v_and_g.split('|')
votes = x[0].split(':')
votes[1]'''

"votes ={}\ngross = {}\nv_and_g = movies_container[0].find(class_='sort-num_votes-visible').get_text(strip=True)\nx= v_and_g.split('|')\nvotes = x[0].split(':')\nvotes[1]"

In [4]:
DataSet = []
number_of_moives = len(movies_container)
i = 0
while(i<number_of_moives):
    movies = {}
    m = movies_container[i]
    try:
        movies['Name'] = m.find(class_='lister-item-header').get_text(strip=True)
    except AttributeError:
        moves['Name'] = None
    try:
        movies['Certificate'] = m.find(class_='certificate').get_text(strip=True)
    except AttributeError:
        movies['Certificate'] = None
    try:
        movies['Runtime'] = m.find(class_='runtime').get_text(strip=True)
    except AttributeError:
        movies['Runtime'] = None
    try:
        movies['Genre'] = m.find(class_='genre').get_text(strip=True)
    except AttributeError:
        movies['Genre'] = None
    try:
        movies['Rating'] = m.find(class_='inline-block ratings-imdb-rating').get_text(strip=True)
    except AttributeError:
        movies['Rating'] = None
    try:
        movies['Metascore'] = m.find(class_='inline-block ratings-metascore').get_text(strip=True)
    except AttributeError:
        movies['Metascore'] = None
    try:
        votes_and_gross = m.find(class_='sort-num_votes-visible').get_text(strip=True)
        v= votes_and_gross.split('|')
        n = v[0].split(':')#
        movies['Votes'] = n[1]
    except AttributeError:
        movies['Votes'] = None
    i = i + 1
    DataSet.append(movies)

dataframe = pd.DataFrame(DataSet)
dataframe['Name'] = dataframe['Name'].astype(str).str[2:]
dataframe['Metascore'] = dataframe['Metascore'].astype(str).str[0:2]

In [7]:
col_names = ["Name","Certificate",'Runtime',"Genre","Rating",'Metascore',"Votes"]
dataframe = dataframe.reindex(columns=col_names)
dataframe.to_csv("Output.csv")
dataframe

,Name,Certificate,Runtime,Genre,Rating,Metascore,Votes
0,Alita: Battle Angel(2019),None,122 min,"Action, Adventure, Romance",7.6,54,"50,123"
1,Polar(2019),None,118 min,"Action, Crime",6.3,19,"41,138"
2,Overlord(2018),R,110 min,"Action, Adventure, Horror",6.8,60,"38,082"
3,Aquaman(2018),PG-13,143 min,"Action, Adventure, Fantasy",7.3,55,"171,894"
4,Incredibles 2(2018),PG,118 min,"Animation, Action, Adventure",7.8,80,"180,659"
5,Robin Hood(I) (2018),PG-13,116 min,"Action, Adventure, Thriller",5.3,32,"28,586"
6,Mortal Engines(2018),PG-13,128 min,"Action, Adventure, Fantasy",6.2,44,"43,251"
7,Spider-Man: Into the Spider-Verse(2018),PG,117 min,"Animation, Action, Adventure",8.6,87,"112,443"
8,Avengers: Infinity War(2018),PG-13,149 min,"Action, Adventure, Fantasy",8.5,68,"590,262"
9,.Black Panther(2018),PG-13,134 min,"Action, Adventure, Sci-Fi",7.3,88,"466,455"
